In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Define your custom functions
def dice_coef(y_true, y_pred, smooth=1):
    intersection = tf.keras.backend.sum(y_true * y_pred, axis=[1,2,3])
    union = tf.keras.backend.sum(y_true, axis=[1,2,3]) + tf.keras.backend.sum(y_pred, axis=[1,2,3])
    return tf.keras.backend.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def dice_loss(in_gt, in_pred):
    return 1 - dice_coef(in_gt, in_pred)

# Load the model
model_path = '/content/drive/My Drive/model.h5'
model = tf.keras.models.load_model(model_path, custom_objects={'dice_loss': dice_loss, 'dice_coef': dice_coef})

def preprocess_image(img):
    img = img.resize((256, 256))  # Adjust target size according to your model
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Assuming your model expects input in range [0, 1]
    return img_array

def predict_deforestation(model, img):
    img_array = preprocess_image(img)
    predictions = model.predict(img_array)
    return predictions

def display_prediction(img, prediction, title, deforestation_rate):
    pred_img_array = prediction[0]  # Assuming the prediction is an image array
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(img)
    axes[0].set_title('Original Image')
    axes[0].axis('off')

    axes[1].imshow(pred_img_array)
    axes[1].set_title(f"{title}\nDeforestation Rate: {deforestation_rate:.2f}%")
    axes[1].axis('off')

    st.pyplot(fig)

def calculate_deforestation_rate(prediction):
    # The deforestation rate is the complement of the afforestation rate
    return (1 - np.mean(prediction)) * 100  # Convert to percentage

def login_page():
    st.title("Deforestation Prediction")
    st.write("## Please Login")
    st.write("")
    st.write("### Enter your credentials below:")

    # Login form
    with st.form(key='login_form'):
        username = st.text_input("Username")
        password = st.text_input("Password", type='password')
        login_button = st.form_submit_button("Login")

    # Dummy authentication (Replace with actual MongoDB authentication)
    if login_button:
        if username == "renvil@test" and password == "password123":  # Dummy credentials
            st.session_state['authenticated'] = True
        else:
            st.error("Invalid username or password")

def main_page():
    st.title("Deforestation Prediction")

    st.write(
        """
        This application uses a deep learning model to predict deforestation from uploaded images.
        Upload two images: one from 10 years ago and one current, and the model will classify and compare the results.
        """
    )

    col1, col2 = st.columns(2)

    with col1:
        uploaded_file_old = st.file_uploader("Choose an image from 10 years ago...", type=["jpg", "jpeg", "png"], key="old_image")

    with col2:
        uploaded_file_new = st.file_uploader("Choose a current image...", type=["jpg", "jpeg", "png"], key="new_image")

    if uploaded_file_old is not None and uploaded_file_new is not None:
        img_old = Image.open(uploaded_file_old)
        img_new = Image.open(uploaded_file_new)

        st.subheader("Uploaded Images")
        col1, col2 = st.columns(2)

        with col1:
            st.image(img_old, caption='Image from 10 years ago', use_column_width=True)

        with col2:
            st.image(img_new, caption='Current Image', use_column_width=True)

        st.write("")
        st.write("Classifying...")

        with st.spinner('Processing the images...'):
            predictions_old = predict_deforestation(model, img_old)
            predictions_new = predict_deforestation(model, img_new)

        st.success('Classification completed!')

        deforestation_rate_old = calculate_deforestation_rate(predictions_old)
        deforestation_rate_new = calculate_deforestation_rate(predictions_new)

        st.subheader("Prediction Results")
        col1, col2 = st.columns(2)

        with col1:
            display_prediction(img_old, predictions_old, 'Predicted Image from 10 years ago', deforestation_rate_old)

        with col2:
            display_prediction(img_new, predictions_new, 'Predicted Current Image', deforestation_rate_new)

        st.write(f"Deforestation Rate 10 years ago: {deforestation_rate_old:.2f}%")
        st.write(f"Current Deforestation Rate: {deforestation_rate_new:.2f}%")

        change_rate = deforestation_rate_new - deforestation_rate_old
        if change_rate > 0:
            st.write(f"Deforestation has increased by: {change_rate:.2f}%")
        else:
            st.write(f"Deforestation has decreased by: {abs(change_rate):.2f}%")

        st.subheader("Analysis and Justification")
        if change_rate > 0:
            st.write(
                f"""
                Based on the predictions, deforestation has increased by {change_rate:.2f}% over the past 10 years.
                This indicates a significant loss of forest cover in the area depicted in the images.
                Possible reasons for this increase could be logging, agricultural expansion, or urban development.
                The impact of increased deforestation includes loss of biodiversity, changes in the ecosystem, and contribution to climate change.
                """
            )
        else:
            st.write(
                f"""
                Based on the predictions, deforestation has decreased by {abs(change_rate):.2f}% over the past 10 years.
                This suggests that there has been an effort to conserve or restore the forest cover in the area depicted in the images.
                Possible reasons for this decrease could be reforestation projects, conservation efforts, or changes in land use policies.
                The benefits of decreased deforestation include preservation of biodiversity, stability of ecosystems, and mitigation of climate change effects.
                """
            )
    else:
        st.info("Please upload both images to start the classification process.")

# Main logic
if 'authenticated' not in st.session_state or not st.session_state['authenticated']:
    login_page()
else:
    main_page()

Overwriting app.py


In [ ]:
!pip install streamlit
!pip install pyngrok
!ngrok config add-authtoken 2jaIIbYzNUO2h3l7GXFSZ08ID7v_6YGEG9FxZXf5ztp7qvyY2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import subprocess
from pyngrok import ngrok

# Kill all existing ngrok tunnels
os.system('pkill ngrok')

# Start Streamlit app
subprocess.Popen(['streamlit', 'run', 'app.py'])

# Establish a new ngrok tunnel
public_url = ngrok.connect(8501)
print(public_url)

NgrokTunnel: "https://d0b2-34-23-151-184.ngrok-free.app" -> "http://localhost:8501"
